# Multiple Calculations
Do all sub-folders. Note that no results are shown.

In [ ]:
# imports
import io_files as io
import fit_mir as fm
import time
import os
import os.path
import json
import scipy.ndimage

In [ ]:
### Load settings

In [ ]:
param = {}
# set the settings file path
param["s_name"] = "settings.json"
param["s_folder"] = r"C:\projects\github\MultipleImageRadiography"
# read the content
path_name = os.path.join(param["s_folder"], param["s_name"])
with open(path_name, "r") as f:
    param.update(json.load(f))
print(param)

In [ ]:
# Do the work
def calc_all(param):
    start_all = time.time()
    time_each = []
    output = io.Text_Output()
    param["output"] = output
    for data in param["data_dirs"]:
        start_each = time.time()
        output.showMessage("Now working on {}.".format(data))
        param["data_dir"] = data
        io.create_paths(param, output)
        image, flat_image, flat2_image = io.load_data(param, param["crop"])
        output.showMessage(
            "No of images loaded are: {}, {}, {}. All should have the same value.".format(
                len(image), len(flat_image), len(flat2_image)
            )
        )
        result1, result2 = fm.fit_dirs(
            param, output, image, flat_image, flat2_image, param["threshold"]
        )
        _filter = scipy.ndimage.median_filter
        param["image_filter"] = _filter
        io.save_results(param, result1, result2)
        end_each = time.time()
        time_each.append(end_each - start_each)
    end_all = time.time()
    each_ave = (sum(time_each) / len(time_each)) / 60.0
    total = (end_all - start_all) / 60.0
    output.showMessage(
        "Finished all folders.\nTotal Time = {:6.2f} min, time per folder = {:6.2f} min.".format(
            total, each_ave
        )
    )


chk_dir = [
    os.path.exists(os.path.join(param["root_dir"], sub_dir))
    for sub_dir in param["data_dirs"]
]
if all(chk_dir):
    calc_all(param)
else:
    print("Not all data folders were found. Check input.")